<a href="https://colab.research.google.com/github/TakehikoEsaka/tips/blob/master/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pandasのドキュメント：<br>

https://pandas.pydata.org/pandas-docs/stable/index.html

In [2]:
import pandas as pd

In [3]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
chipo = pd.read_csv(url, sep = '\t')

忘れかけてた操作

In [6]:
# ブロードキャスト働いて値を一括に入れることが可能
c = chipo.copy()
c["sample"] = "Hey"
c.head()

,order_id,quantity,item_name,choice_description,item_price,sample
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,Hey
1,1,1,Izze,[Clementine],$3.39,Hey
2,1,1,Nantucket Nectar,[Apple],$3.39,Hey
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,Hey
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,Hey


In [11]:
c = [row for row in chipo["item_name"] if "Salsa" in row]
d = chipo[chipo["item_name"].isin(c)]
d.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
15,8,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
25,13,1,Chips and Fresh Tomato Salsa,NaN,$2.39
30,15,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39


中の情報を見る

In [ ]:
chipo.info() # entries <= 4622 observations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [ ]:
#　統計値を見る
chipo.describe()

,order_id,quantity
count,4622.000000,4622.000000
mean,927.254868,1.075725
std,528.890796,0.410186
min,1.000000,1.000000
25%,477.250000,1.000000
50%,926.000000,1.000000
75%,1393.000000,1.000000
max,1834.000000,15.000000


In [ ]:
# 一意の値を見る
# Seriesにしか使えない
chipo["order_id"].unique()

array([   1,    2,    3, ..., 1832, 1833, 1834])

In [ ]:
# カウントはSeriesもDataFrameもどちらも使える
chipo["order_id"].count()

chipo.count()

order_id              4622
quantity              4622
item_name             4622
choice_description    4622
item_price            4622
dtype: int64

In [ ]:
# カウントはSeriesにしか使えない
chipo["order_id"].value_counts()

926     23
1483    14
205     12
691     11
1786    11
        ..
105      1
702      1
718      1
568      1
800      1
Name: order_id, Length: 1834, dtype: int64

apply


groupbyを使う

In [ ]:
# as_indexはグループをインデックスにするかしないか
c = chipo.groupby('item_name', as_index=True)
# このままの状態だとcはgroupbyオブジェクトのまま．以下のようにaggと併せて使う

In [ ]:
c = c.agg(['count', 'mean', 'max', 'min', 'sum'])

除去

In [ ]:
# 複製除去
print(chipo.shape)
c = chipo.drop_duplicates(['item_name','quantity','choice_description'])
print(c.shape)

(4622, 5)
(1949, 5)


In [ ]:
# 値除去
# columnsをつけないと認識してくれない
chipo.drop(columns = ["item_name"]).shape

(4622, 4)

In [ ]:
# delを使って削除
try:
  del c['item_name']
except:
  pass
c.head()

,order_id,quantity,choice_description,item_price
100,1,1,0,$2.39
101,1,1,[Clementine],$3.39
102,1,1,[Apple],$3.39
103,1,1,0,$2.39
104,2,2,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [ ]:
# 欠損値を落とす
# howは"all"か"any"を選べる．threshを設定して何個以上あればNanをとるという方法も可能
c = chipo.dropna(how='any')
c.head()

,order_id,quantity,item_name,choice_description,item_price
100,1,1,Chips and Fresh Tomato Salsa,0,$2.39
101,1,1,Izze,[Clementine],$3.39
102,1,1,Nantucket Nectar,[Apple],$3.39
103,1,1,Chips and Tomatillo-Green Chili Salsa,0,$2.39
104,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


ソート

In [ ]:
# インデックスでソート
c = chipo.sort_index()
c.head()

,order_id,quantity,item_name,choice_description,item_price
100,1,1,Chips and Fresh Tomato Salsa,0,$2.39
101,1,1,Izze,[Clementine],$3.39
102,1,1,Nantucket Nectar,[Apple],$3.39
103,1,1,Chips and Tomatillo-Green Chili Salsa,0,$2.39
104,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [ ]:
# 昇順か降順はascendingで調節出来る
c = chipo.sort_values(by = ['quantity'], ascending=False)
c.head()

,order_id,quantity,item_name,choice_description,item_price
3698,1443,15,Chips and Fresh Tomato Salsa,0,$44.25
4252,1660,10,Bottled Water,0,$15.00
3987,1559,8,Side of Chips,0,$13.52
3699,1443,7,Bottled Water,0,$10.50
2541,970,5,Bottled Water,0,$7.50


In [ ]:
chipo["order_id"]

100        1
101        1
102        1
103        1
104        2
        ... 
4717    1833
4718    1833
4719    1834
4720    1834
4721    1834
Name: order_id, Length: 4622, dtype: int64

In [ ]:
# todo ：ラムダ式で並び替える
# もうすこし複雑な処理をしたい時はpython組み込み関数を使う
c = sorted(chipo, key = chipo["order_id"].index, reverse = True)

TypeError: ignored

インデックスを振り直す

In [ ]:
old_indexes = chipo.index
c.index = [i for i in range(1, 1 + chipo.shape[0], 1)]

,order_id,quantity,item_name,choice_description,item_price
2,1660,10,Bottled Water,0,$15.00


In [ ]:
# あるカラムをindexにしてしまう
c = chipo.set_index("item_name")
c.head()

,order_id,quantity,choice_description,item_price
item_name,,,,
Chips and Fresh Tomato Salsa,1,1,NaN,$2.39
Izze,1,1,[Clementine],$3.39
Nantucket Nectar,1,1,[Apple],$3.39
Chips and Tomatillo-Green Chili Salsa,1,1,NaN,$2.39
Chicken Bowl,2,2,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [ ]:
#drop=True を指定しなければ、旧インデックスがデータ列に移動します。
# インデックスの値を指定しない場合は０から振られる
c = chipo.reset_index(drop = True)
c.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,0,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,0,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [ ]:
 # datatimeをindexにつける (時系列データのresmapleとかが使える)
 pd.DatetimeIndex(chipo["order_id"])
 test.index = pd.DatetimeIndex(test2[0])

DatetimeIndex(['1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000002',
               '1970-01-01 00:00:00.000000003',
               '1970-01-01 00:00:00.000000003',
               '1970-01-01 00:00:00.000000004',
               '1970-01-01 00:00:00.000000004',
               '1970-01-01 00:00:00.000000005',
               ...
               '1970-01-01 00:00:00.000001831',
               '1970-01-01 00:00:00.000001831',
               '1970-01-01 00:00:00.000001831',
               '1970-01-01 00:00:00.000001832',
               '1970-01-01 00:00:00.000001832',
               '1970-01-01 00:00:00.000001833',
               '1970-01-01 00:00:00.000001833',
               '1970-01-01 00:00:00.000001834',
               '1970-01-01 00:00:00.000001834',
               '1970-01-01 00:00:00.000001834'],
              dtype=

欠損値の処理

In [ ]:
c["choice_description"] = c["choice_description"].fillna(0)

相関係数を計算

In [ ]:
# 相関係数ヒートマップを計算する時
chipo.corr()

,order_id,quantity
order_id,1.000000,0.032397
quantity,0.032397,1.000000


In [ ]:
# 単体の特徴量に対して計算する時
chipo["order_id"].corr(chipo["quantity"])

0.032396894926411623

リネーム＆マージ

In [ ]:
# on
df_ab = pd.DataFrame({'a': ['a_1', 'a_2', 'a_3'], 'b': ['b_1', 'b_2', 'b_3']})
df_ac = pd.DataFrame({'a': ['a_1', 'a_2', 'a_4'], 'c': ['c_1', 'c_2', 'c_4']})
print(pd.merge(df_ab, df_ac, on='a'))

     a    b    c
0  a_1  b_1  c_1
1  a_2  b_2  c_2


In [ ]:
# リネーム : 辞書型で指定
df_ac_ = df_ac.rename(columns = {"a": "a_"})

In [ ]:
# right_on, left_on
c = pd.merge(df_ab, df_ac_, left_on="a", right_on="a_")

In [ ]:
# how : "inner(内部結合)"，"left（左結合）", "right（右結合）"，"outer（外部結合）"
# 以下のhowの所を変えると様子分かる
c = pd.merge(df_ab, df_ac, how = "outer")
c.head()

,a,b,c
0,a_1,b_1,c_1
1,a_2,b_2,c_2
2,a_3,b_3,NaN
3,a_4,NaN,c_4


サンプル

In [ ]:
# ランダムサンプル
chipo.sample(n=5)



,order_id,quantity,item_name,choice_description,item_price
2878,1143,1,Canned Soft Drink,[Coke],$1.25
246,108,1,Canned Soda,[Mountain Dew],$1.09
3497,1405,1,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$8.49
4370,1745,1,Chips and Guacamole,NaN,$4.45
2055,828,1,Chips and Fresh Tomato Salsa,NaN,$2.39


文字列メソッド

In [ ]:
# Seriesに専用のstrを使うと出来る．
c = chipo["item_price"].str.replace("$", "").astype('float32')
c.head()

0     2.39
1     3.39
2     3.39
3     2.39
4    16.98
Name: item_price, dtype: float32

In [ ]:
# expand設定がなければ通常のリストが入ったSeriesが生成
c = chipo["item_price"].str.split(".")
c.head()

# splitにexpand = Trueをつけると複製されてDataframeが出来る
c = chipo["item_price"].str.split(".", expand = True)
c.head()

,0,1
0,$2,39
1,$3,39
2,$3,39
3,$2,39
4,$16,98


クリップ

In [ ]:
c = chipo.copy()
c["item_price"] = chipo["item_price"].str.replace("$", "").astype("float32")
c["item_price"].clip(3,10).head()

0     3.00
1     3.39
2     3.39
3     3.00
4    10.00
Name: item_price, dtype: float32

リサンプル
- 時系列データに有効．但しdataframe_indexにしか使えない

In [ ]:
chipo.resample("W")

TypeError: ignored

ピボット

In [ ]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})

In [ ]:
# 指定したindex, columns，valuesでテーブルをreshapeする
c = df.pivot(index='foo', columns='bar', values='baz')
c.head()

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


メルト

In [ ]:
# tidy_data :　整然データ
# 参考：https://qiita.com/ishida330/items/922caa7acb73c1540e28

todo
- apply
https://github.com/guipsamora/pandas_exercises/blob/master/04_Apply/Students_Alcohol_Consumption/Exercises_with_solutions.ipynb <br>

- loc, iloc, at

- get_dummies(drop_first = True)

- assign

- transformm(lamda使った方法と併せて)

- isnull

- to_csv

- cumsum, cummin

- pd.to_dataframe()

In [ ]:
# pd.Seriesの速い作り方（zipとdictを使う）
mylist = list('abcedfghijklmnopqrstuvwxyz')
myarr = np.arange(26)
mydict = dict(zip(mylist, myarr))
ser = pd.Series(mydict)
print(ser[:5])

NameError: ignored

isin

ser1 = pd.Series([1, 2, 3, 4, 5])
ser2 = pd.Series([4, 5, 6, 7, 8])
ser1[~ser1.isin(ser2)]

In [ ]:
qcut : binで区切る

d.qcut(ser, q = 10)

In [ ]:
whereと組み合わせた使い方
ser.where(lambda x: x%3 == 0).dropna()

In [ ]:
mapとseriesを組み合わせた使い方



In [ ]:
rolling / diff((periods = 1)と併せた使い方

NameError: ignored

In [ ]:
# parseとmapを組み合わせた使い方

ser = pd.Series(['Jan 2010', 'Feb 2011', 'Mar 2012'])

# solution using parser
from dateutil.parser import parse
ser.map(lambda x: parse('04 ' + x))

# another solution

from dateutil.parser import parse
# Parse the date
ser_ts = ser.map(lambda x: parse(x))

# Construct date string with date as 4
ser_datestr = ser_ts.dt.year.astype('str') + '-' + ser_ts.dt.month.astype('str') + '-' + '04'

# Format it.
[parse(i).strftime('%Y-%m-%d') for i in ser_datestr]

counterと併せた使い方
ser = pd.Series(['Apple', 'Orange', 'Plan', 'Python', 'Money'])


from collections import Counter
mask = ser.map(lambda x: sum([Counter(x.lower()).get(i, 0) for i in list('aeiou')]) >= 2)
ser[mask]

# skipして良し
# 正規表現と併せた使い方
# input
emails = pd.Series(['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com'])
pattern ='[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,4}'
# using powerful regex
import re
re_ = re.compile(pattern)
emails[emails.str.contains(pat = re_, regex = True)]

# other solutions
pattern ='[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,4}'
mask = emails.map(lambda x: bool(re.match(pattern, x)))
emails[mask]

# using str.findall
emails.str.findall(pattern, flags=re.IGNORECASE)

# using list comprehension
[x[0] for x in [re.findall(pattern, email) for email in emails] if len(x) > 0]

In [ ]:
pandasのautocorrelationで自己相関を計算

